# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

## Лабораторная работа 9

In [50]:
import pandas as pd

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [2]:
import nltk
text = 'Tokenizers divide strings into lists of substrings. For example, tokenizers can be used to find the words and punctuation in a string.'
words = list(set(nltk.word_tokenize(text)))
words

['string',
 'can',
 'the',
 'substrings',
 'tokenizers',
 'divide',
 'in',
 'example',
 'and',
 'Tokenizers',
 'into',
 'a',
 'punctuation',
 'be',
 'strings',
 'lists',
 'words',
 '.',
 'to',
 'For',
 'find',
 ',',
 'of',
 'used']

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [10]:
import random
from nltk.metrics import *
lst = [0] * 5
for i in range(5):
    lst[i] = random.sample(words, 2)
    print(f'Пара: {lst[i]}, расстояние редактирования: {edit_distance(lst[i][0], lst[i][1])}')

Пара: ['to', 'tokenizers'], расстояние редактирования: 8
Пара: ['a', 'find'], расстояние редактирования: 4
Пара: ['Tokenizers', 'words'], расстояние редактирования: 8
Пара: ['used', ','], расстояние редактирования: 4
Пара: [',', 'string'], расстояние редактирования: 6


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [12]:
def find_words(word, k):
    dct = {w: edit_distance(w, word) for w in words}
    dct = sorted(dct, key = dct.get)[:k]
    return dct
print(find_words('divide', 5))

['divide', 'find', 'lists', 'in', 'and']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [10]:
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer(language='english')
df = pd.DataFrame({'stemmed_word': [stemmer.stem(word) for word in words], 'normalized_word': [lemmatizer.lemmatize(word) for word in words]}, index=words)
df

,stemmed_word,normalized_word
string,string,string
can,can,can
the,the,the
substrings,substr,substring
tokenizers,token,tokenizers
divide,divid,divide
in,in,in
example,exampl,example
and,and,and
Tokenizers,token,Tokenizers


In [ ]:
nltk.download('stopwords')

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [16]:
from nltk.corpus import stopwords
text = 'Tokenizers divide strings into lists of substrings. For example, tokenizers can be used to find the words and punctuation in a string.'
tokens = word_tokenize(text.lower())
english_stopwords = stopwords.words('english')
tokens_without_stopwords = [word for word in tokens if word not in english_stopwords]
tokens_without_stopwords

['tokenizers',
 'divide',
 'strings',
 'lists',
 'substrings',
 '.',
 'example',
 ',',
 'tokenizers',
 'used',
 'find',
 'words',
 'punctuation',
 'string',
 '.']

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
recipes = recipes.head()
vectors  = vectorizer.fit_transform(recipes['description'])
# print(vectorizer.vocabulary_)
k = vectors.shape[0]
for vector in range(k):
    print(vectors[vectors].toarray())

IndexError: Indexing with sparse matrices is not supported except boolean indexing where matrix and index are equal shapes.

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [62]:
from scipy.spatial import distance
import numpy
import pandas as pd
lst =[]
for el in recipes['description']:
    lst.append(word_tokenize(el))
vectors  = vectorizer.fit_transform(recipes['description']).toarray()
for index_1 in range(len(lst)):
    for index_2 in range(index_1 + 1, len(lst)):
        print(distance.cosine(vectors[index_1], vectors[index_2]))

0.9725198666999536
0.8944228631111404
0.8408701219963103
0.8093525908759086
1.0
0.8551689750098787
0.9214042717289879
0.9640049285228507
0.9529584291886545
0.8788072331736545
